In [1]:
import pandas as pd 
import numpy as np 
import time, gc, os 
from tqdm import tqdm as tqdm
import logging
import gensim
from gensim.models import word2vec

In [2]:
# !pip install gensim
!nvidia-smi

Sat Jul  4 02:35:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           On   | 00000000:00:08.0 Off |                    0 |
| N/A   43C    P0    57W / 250W |      0MiB / 22919MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P40           On   | 00000000:00:09.0 Off |                    0 |
| N/A   40C    P0    56W / 250W |      0MiB / 22919MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [4]:
ad_df = pd.read_pickle("data/all_ad.pkl")

In [5]:
print(ad_df.dtypes)
ad_df.head()

creative_id         object
ad_id               object
product_id          object
product_category    object
advertiser_id       object
industry            object
dtype: object


,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,0,5,381,78
1,4,4,0,5,108,202
2,7,7,0,5,148,297
3,8,8,0,5,713,213
4,9,9,0,5,695,213


# cid embedding矩阵

In [12]:
cid2id = dict(zip(ad_df['creative_id'].unique(), range(2, ad_df['creative_id'].nunique() + 2)))
print(len(cid2id)) 

model = word2vec.Word2Vec.load('data/word2vec_model/word2vec_cid_300d.model')
embed_size = 300
weight = np.zeros((len(cid2id) + 2, embed_size), dtype=np.float32)

for x in tqdm(cid2id.items()):
    try:
        weight[x[1], :] = model[x[0]]
    except:
        continue
weight[1, :] = weight[2:].mean(axis=0)
np.save('data/cid_weight_300d.npy', weight)

4445720


100%|██████████| 4445720/4445720 [00:44<00:00, 99480.77it/s] 


In [20]:
!du -Sh data/

20G	data/word2vec_model
4.0K	data/.ipynb_checkpoints
6.2G	data/


# ader embedding矩阵

In [21]:
ader2id = dict(zip(ad_df['advertiser_id'].unique(), range(2, ad_df['advertiser_id'].nunique() + 2)))
print(len(ader2id)) 

model = word2vec.Word2Vec.load('data/word2vec_model/word2vec_ader_300d.model')
embed_size = 300
weight = np.zeros((len(ader2id) + 2, embed_size), dtype=np.float32)

for x in tqdm(ader2id.items()):
    try:
        weight[x[1], :] = model[x[0]]
    except:
        continue
weight[1, :] = weight[2:].mean(axis=0)
np.save('data/ader_weight_300d.npy', weight)

62965


100%|██████████| 62965/62965 [00:00<00:00, 103075.89it/s]


# adid embedding矩阵

In [22]:
adid2id = dict(zip(ad_df['ad_id'].unique(), range(2, ad_df['ad_id'].nunique() + 2)))
print(len(adid2id)) 

model = word2vec.Word2Vec.load('data/word2vec_model/word2vec_adid_300d.model')
embed_size = 300
weight = np.zeros((len(adid2id) + 2, embed_size), dtype=np.float32)

for x in tqdm(adid2id.items()):
    try:
        weight[x[1], :] = model[x[0]]
    except:
        continue
weight[1, :] = weight[2:].mean(axis=0)
np.save('data/adid_weight_300d.npy', weight)

3812202


100%|██████████| 3812202/3812202 [00:38<00:00, 99473.96it/s] 


# pid embedding矩阵

In [26]:
pid2id = dict(zip(ad_df['product_id'].unique(), range(2, ad_df['product_id'].nunique() + 2)))
print(len(pid2id)) 

model = word2vec.Word2Vec.load('data/word2vec_model/word2vec_pid_300d.model')
embed_size = 300
weight = np.zeros((len(pid2id) + 2, embed_size), dtype=np.float32)

for x in tqdm(pid2id.items()):
    try:
        weight[x[1], :] = model[x[0]]
    except:
        continue
weight[1, :] = weight[2:].mean(axis=0)
np.save('data/pid_weight_300d.npy', weight)

44315


100%|██████████| 44315/44315 [00:00<00:00, 102892.56it/s]


# 得到nn模型所需的id文件

In [40]:
# %%time

# maxlen = 90

# train_xlen = [0]*3000000
# with open("data/user_creative_ids_train.txt", encoding='utf-8') as f:
#     for i, line in tqdm(enumerate(f)):
#         x = line.split("\t")
#         ids = x[1]
#         x = ids.split()
#         train_xlen[i] = len(x)
# train_xlen = np.array(train_xlen)
# print(pd.Series(train_xlen).describe())
# print(np.percentile(train_xlen, 99))

In [2]:
import pandas as pd
ad_df = pd.read_pickle("data/all_ad.pkl")
ad_df.head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,0,5,381,78
1,4,4,0,5,108,202
2,7,7,0,5,148,297
3,8,8,0,5,713,213
4,9,9,0,5,695,213


In [3]:
for col in ad_df.columns:
    print(col, ad_df[col].nunique())

creative_id 4445720
ad_id 3812202
product_id 44315
product_category 18
advertiser_id 62965
industry 336


# 广告类别转换

In [4]:
cid2ader = dict(zip(ad_df["creative_id"], ad_df["advertiser_id"]))
cid2adid = dict(zip(ad_df["creative_id"], ad_df["ad_id"]))
cid2pid = dict(zip(ad_df["creative_id"], ad_df["product_id"]))

cid2pcat = dict(zip(ad_df["creative_id"], ad_df["product_category"]))
cid2ind = dict(zip(ad_df["creative_id"], ad_df["industry"]))

In [5]:
cid2id = dict(zip(ad_df['creative_id'].unique(), range(2, ad_df['creative_id'].nunique() + 2)))
ader2id = dict(zip(ad_df['advertiser_id'].unique(), range(2, ad_df['advertiser_id'].nunique() + 2)))
adid2id = dict(zip(ad_df['ad_id'].unique(), range(2, ad_df['ad_id'].nunique() + 2)))
pid2id = dict(zip(ad_df['product_id'].unique(), range(2, ad_df['product_id'].nunique() + 2)))

pcat2id = dict(zip(ad_df['product_category'].unique(), range(2, ad_df['product_category'].nunique() + 2)))
ind2id = dict(zip(ad_df['industry'].unique(), range(2, ad_df['industry'].nunique() + 2)))

In [6]:
print(len(cid2id)+2, len(ader2id)+2, len(adid2id)+2, len(pid2id)+2, len(pcat2id)+2, len(ind2id)+2)

4445722 62967 3812204 44317 20 338


In [7]:
# 读取user数据（标签数据）
user = pd.read_csv("data/all_user.csv")
print(user.shape)
uid2age = dict(zip(user['user_id'], user['age']))
uid2gender = dict(zip(user['user_id'], user['gender']))

(3000000, 3)


# 训练集处理

In [8]:
%%time

maxlen = 150

train_cid = []
train_ader = []
train_pid = []
train_adid = []
train_pcat = []
train_ind = []

train_age = []
train_gender = []

with open("data/user_creative_ids_train.txt", encoding='utf-8') as f:
    for line in tqdm(f):
        x = line.split("\t")
        uid = int(x[0])  # user_id
        cid = x[1].split()   # cid_sequence
        # cid
        x_cid = [cid2id.get(i, 1) for i in cid]
        
        # ader
        x_ader = [cid2ader.get(i, '0') for i in cid]
        x_ader = [ader2id.get(i, 1) for i in x_ader]
        
        # adid
        x_adid = [cid2adid.get(i, '0') for i in cid]
        x_adid = [adid2id.get(i, 1) for i in x_adid]
        
        # pid
        x_pid = [cid2pid.get(i, '0') for i in cid]
        x_pid = [pid2id.get(i, 1) for i in x_pid]
        
        # pcat
        x_pcat = [cid2pcat.get(i, '0') for i in cid]
        x_pcat = [pcat2id.get(i, 1) for i in x_pcat]
        
        # ind 
        x_ind = [cid2ind.get(i, '0') for i in cid]
        x_ind = [ind2id.get(i, 1) for i in x_ind]
        
        assert len(x_cid) == len(x_ader) == len(x_adid) == len(x_pid) == len(x_pcat) == len(x_ind)
        
        if len(cid) >= maxlen:
            x_cid = x_cid[:maxlen]
            x_ader = x_ader[:maxlen]
            x_adid = x_adid[:maxlen]
            x_pid = x_pid[:maxlen]
            x_pcat = x_pcat[:maxlen]
            x_ind = x_ind[:maxlen]
            
        elif len(cid) < maxlen:
            x_cid = x_cid + [0] * (maxlen - len(cid))
            x_ader = x_ader + [0] * (maxlen - len(cid))
            x_adid = x_adid + [0] * (maxlen - len(cid))
            x_pid = x_pid + [0] * (maxlen - len(cid))
            x_pcat = x_pcat + [0] * (maxlen - len(cid))
            x_ind = x_ind + [0] * (maxlen - len(cid))

        train_cid.append(x_cid)
        train_ader.append(x_ader)
        train_adid.append(x_adid)
        train_pid.append(x_pid)
        train_pcat.append(x_pcat)
        train_ind.append(x_ind)
        
        train_age.append(uid2age[uid])
        train_gender.append(uid2gender[uid])

train_cid = np.array(train_cid, dtype=np.int32)
train_ader = np.array(train_ader, dtype=np.int32)   # 广告主id序列
train_adid = np.array(train_adid, dtype=np.int32)
train_pid = np.array(train_pid, dtype=np.int32)
train_pcat = np.array(train_pcat, dtype=np.int32)
train_ind = np.array(train_ind, dtype=np.int32)

train_age = np.array(train_age, dtype=np.int32) - 1
train_gender = np.array(train_gender, dtype=np.int32) - 1

print(train_cid.shape, train_ader.shape, train_adid.shape, train_pid.shape, 
      train_pcat.shape, train_ind.shape, 
      train_age.shape, train_gender.shape)

3000000it [12:34, 3977.40it/s]


(3000000, 150) (3000000, 150) (3000000, 150) (3000000, 150) (3000000, 150) (3000000, 150) (3000000,) (3000000,)
CPU times: user 16min 37s, sys: 18.3 s, total: 16min 56s
Wall time: 16min 52s


In [9]:
np.save("data/nn_data/train_pcat.npy", train_pcat)
np.save("data/nn_data/train_ind.npy", train_ind)

In [10]:
np.save("data/nn_data/train_cid.npy", train_cid)
np.save("data/nn_data/train_ader.npy", train_ader)
np.save("data/nn_data/train_adid.npy", train_adid)
np.save("data/nn_data/train_pid.npy", train_pid)

np.save("data/nn_data/train_age.npy", train_age)
np.save("data/nn_data/train_gender.npy", train_gender)

# 测试集处理

In [11]:
maxlen = 150

test_cid = []
test_ader = []
test_pid = []
test_adid = []
test_pcat = []
test_ind = []


with open("data/user_creative_ids_test.txt", encoding='utf-8') as f:
    for line in tqdm(f):
        x = line.split("\t")
        uid = int(x[0])  # user_id
        cid = x[1].split()   # cid_sequence
        # cid
        x_cid = [cid2id.get(i, 1) for i in cid]
        
        # ader
        x_ader = [cid2ader.get(i, '0') for i in cid]
        x_ader = [ader2id.get(i, 1) for i in x_ader]
        
        # adid
        x_adid = [cid2adid.get(i, '0') for i in cid]
        x_adid = [adid2id.get(i, 1) for i in x_adid]
        
        # pid
        x_pid = [cid2pid.get(i, '0') for i in cid]
        x_pid = [pid2id.get(i, 1) for i in x_pid]
        
        # pcat
        x_pcat = [cid2pcat.get(i, '0') for i in cid]
        x_pcat = [pcat2id.get(i, 1) for i in x_pcat]
        
        # ind 
        x_ind = [cid2ind.get(i, '0') for i in cid]
        x_ind = [ind2id.get(i, 1) for i in x_ind]
        
        assert len(x_cid) == len(x_ader) == len(x_adid) == len(x_pid) == len(x_pcat) == len(x_ind)
        
        if len(cid) > maxlen:
            x_cid = x_cid[:maxlen]
            x_ader = x_ader[:maxlen]
            x_adid = x_adid[:maxlen]
            x_pid = x_pid[:maxlen]
            x_pcat = x_pcat[:maxlen]
            x_ind = x_ind[:maxlen]
            
        else:
            x_cid = x_cid + [0] * (maxlen - len(cid))
            x_ader = x_ader + [0] * (maxlen - len(cid))
            x_adid = x_adid + [0] * (maxlen - len(cid))
            x_pid = x_pid + [0] * (maxlen - len(cid))
            x_pcat = x_pcat + [0] * (maxlen - len(cid))
            x_ind = x_ind + [0] * (maxlen - len(cid))
            
        test_cid.append(x_cid)
        test_ader.append(x_ader)
        test_adid.append(x_adid)
        test_pid.append(x_pid)
        test_pcat.append(x_pcat)
        test_ind.append(x_ind)

test_cid = np.array(test_cid, dtype=np.int32)
test_ader = np.array(test_ader, dtype=np.int32)   # 广告主id序列
test_adid = np.array(test_adid, dtype=np.int32)
test_pid = np.array(test_pid, dtype=np.int32)
test_pcat = np.array(test_pcat, dtype=np.int32)
test_ind = np.array(test_ind, dtype=np.int32)

print(test_cid.shape, test_ader.shape, test_adid.shape, 
      test_pid.shape, test_pcat.shape, test_ind.shape)

1000000it [03:41, 4523.20it/s]


(1000000, 150) (1000000, 150) (1000000, 150) (1000000, 150) (1000000, 150) (1000000, 150)


In [12]:
np.save("data/nn_data/test_pcat.npy", test_pcat)
np.save("data/nn_data/test_ind.npy", test_ind)

In [13]:
np.save("data/nn_data/test_cid.npy", test_cid)
np.save("data/nn_data/test_ader.npy", test_ader)
np.save("data/nn_data/test_adid.npy", test_adid)
np.save("data/nn_data/test_pid.npy", test_pid)